# TensorFlowのインストール
```bash
pip install tensorflow
```
もし使っているマシンがGPUに対応しているならば、下記のコマンドを使ってインストールしておくと、GPU環境でも簡単にTensorFlowを使うことができる。
```bash
pip install tensorflow-gpu
```

# TensorFlowの使い方の基本

In [2]:
# 税込金額を計算するPythonプログラム
RATE = 0.08
price = 1000
tax = RATE * price
price_tax = price + tax

price = price_tax


print('RATE = ', RATE)
print('tax = ', tax)
print('price = ', price)

RATE =  0.08
tax =  80.0
price =  1080.0


計算過程が確認できるように、処理ごとにコードを分けていく。

### `tf.constant()` : 定数のテンソルを作成
```python
tensorflow.constant(
    value,
    dtype=None,
    shape=None,
    name='Const',
    verify_shape=False
)
```

| 引数 | 説明 |
|:------:|:----------------------------------------|
| `value` | 定数値または定数値のリストを指定. 値がリストの場合は、shape引数でサイズ(長さ)を指定する際に、実際のサイズが収まるようにサイズ指定することが必要. リストの長さが指定された要素の数よりも少ない場合は、リストの最後の要素が残りの要素を埋めるために使用される. |
| `dtype` | 型を指定. 指定されていない場合は、valueの値から推論された型がセットされる. |
| `shape` | オプション. データのサイズ(長さ)を指定する |
| `name` | 定数を内部的に識別するための名前. 指定しない場合はデフォルトのConstが使用される |
| `verify_shape` | valueの形状を検証可能にするかどうかを論理値で指定する. デフォルトはFalse(検証可能にしない). |

### `tf.Variable()` : 変数のテンソルを作成する
**<font color="red">※`tf.Variable()`を実行すると以下の初期化メソッドが呼ばれる</font>**.
```python
__init__(
    initial_value=None,
    trainable=True,
    collections=None,
    validate_shape=True,
    caching_device=None,
    name='Variable',
    variable_def=None,
    dtype=None,
    expected_shape=None,
    import_scope=None,
    constraint=None
)
```

| 引数名 | 説明 |
| :------- | :------- |
| `initial_value` | 変数の値として使用できるPythonオブジェクト. 初期値を設定しないことも可能だが、その場合は、dtype指定する必要がある.<br> 値がリストの場合は、shape引数でサイズ(長さ)を指定する際に、実際のサイズが収まるようにサイズ指定することが必要. リストの長さが指定された要素の数よりも少ない場合は、リストの最後の要素が残りの要素を埋めるために使用される. |
| `trainable` | デフォルトのTrueで、変数をグラフに追加する. |
| `collections` | グラフコレクションキーのリストで、変数はこれらのコレクションに追加される. デフォルトのNoneで[GraphKeys.GLOBAL_VARIABLES]が使われる. |
| `validate_shape` | Falseに設定すると、変数が未知の値で初期化されるのを許可する. デフォルトのTrueは、initial_valueの値によって形状を決定する |
| `caching_device` | 変数に読み込む値をキャッシュデータとして保存する場合を指定するためのオプション |
| `name` | 変数を内部的に識別するための名前. 指定しない場合はデフォルトのVariableが使用される |
| `expected_shape` | initial_valueの形状をあらかじめ指定する場合に使用する |
| `import_scope` | 変数のスコープとして使用する文字列を設定するためのオプション. プロトコルバッファから初期化するときにのみ使用する |
| `constraint` | 更新(値をセット)された後に変数に適用される関数を指定するオプション. |

In [4]:
import tensorflow as tf    # TensorFlowのインポート

# データフローグラフの作成
rate = tf.constant(0.08, dtype=tf.float32)    # 消費税率(float32)のopノード
price = tf.Variable(1000.0, dtype=tf.float32)    # 価格(float32)を保持する変数
tax = tf.multiply(rate, price)    # 税額を計算するopノード
price_tax = tf.add(tax, price)    # 税込金額を計算するopノード

# 変数priceを更新するopノード
update = tf.assign(price, price_tax)

In [5]:
#  データフローグラフの各要素を出力
print('rate          = ', rate)
print('price         =', price)
print('tax            =', tax)
print('price_tax = ', price_tax)
print('update     =', update)

rate          =  Tensor("Const_1:0", shape=(), dtype=float32)
price         = <tf.Variable 'Variable_1:0' shape=() dtype=float32_ref>
tax            = Tensor("Mul_1:0", shape=(), dtype=float32)
price_tax =  Tensor("Add_1:0", shape=(), dtype=float32)
update     = Tensor("Assign_1:0", shape=(), dtype=float32_ref)


## データフローグラフって何?
↑これが「データフローグラフ」の実体。データフローグラフは、ノードと呼ばれる計算ブロックのようなものを「エッジ」と呼ばれる線で接続したもので、データはノードからノードへ、エッジのつながり方に従って受け渡され、計算が行われる。

## データフローグラフの実行

### `tf.Session()` : Seesionオブジェクトを生成する
**<font color="red">※`tf.Session()`を実行すると以下の初期化メソッドが呼ばれる</font>**

```python
__init__(
    target='',
    graph=None,
    config=None
)
```

| 引数 | 説明 |
|:-----:|:------|
| `target` | オプション。接続先の実行エンジンを指定する。 |
| `graph` | オプション。使用するデータフローグラフを指定する場合に使用する。 |
| `config` | オプション。セッションの構成オプションを指定する。 |

```python
# セッションスタート
sess = tf.Session()
```

### `Sessionオブジェクト.run(opノード)` : opノード(Tensorオブジェクト)を評価し、計算操作の結果を返す
```python
run(
    fetches,
    feed_dict=None,
    options=None,
    run_metadata=None
)
```

| 引数 | 説明 |
|:-----:|:------|
| fetches | 計算グラフのopノード(Tensorオブジェクト)を指定する。 |
| feed_dict | 型を指定。指定されていない場合はfetchesから推論された型がセットされる。 |
| options | オプション。このオプションを使って特定のステップの振る舞いを制御できる(トレースをオンにするなど)。 |
| run_metadata | オプション。非テンソルからの出力を収集する。 |

### `Variableオブジェクト.initializer` : 変数の初期化
**変数(Variable)の初期化**
```python
# initializerプロパティの戻り値(初期化op)を取得する.
Sessionオブジェクト.run(Variableオブジェクト.initializer)

# 初期化が済むと他のopノードと同じ状態になる.
Sessionオブジェクト.run(Variableオブジェクト)
```

In [6]:
# セッション部
sess = tf.Session()    # セッションスタート
print('rate = ', sess.run(rate))    # rateノードを実行
sess.run(price.initializer)    # 変数priceは初期化が必要
print('price = ', sess.run(price))    # 変数priceを出力

sess.run(update)    # updateノードを実行
print('price(update) = ', sess.run(price))    # 更新後のprice

sess.close()    # セッションを閉じる

rate =  0.08
price =  1000.0
price(update) =  1080.0


# データフローグラフを実行するいくつかのパターン

## `tf.global_variables_initializer()`で変数をまとめて初期化する
`tf.global_variables_initializer()`は、定義済みのすべての変数をまとめて初期化する。

> `tf.global_variables_initializer()` : **すべてのグローバル変数を初期化するための処理を返す。**

```python
Sessionオブジェクト.run(tf.global_variables_initializer())
```

In [7]:
# 例)
import tensorflow as tf

# データフローグラフの作成
rate = tf.constant(0.08, dtype=tf.float32)
price = tf.Variable(1000.0, dtype=tf.float32)
tax = tf.multiply(rate, price)
price_tax = tf.add(tax, price)
update = tf.assign(price, price_tax)

# --- セッション部 ---
sess = tf.Session()    # セッションスタート
print('rate = ', sess.run(rate))    # rateノードを実行
sess.run(tf.global_variables_initializer())
print('price = ', sess.run(price))

sess.run(update)    # updateノードを実行
print('price(update) = ', sess.run(price))    # 更新後のpriceを出力

sess.close()    # セッションクローズ
# --- セッション部 ---

rate =  0.08
price =  1000.0
price(update) =  1080.0


## `tf.InteractiveSession()` : セッションをインタラクティブモードで実行する
<div style="text-align:center">
    Sessionオブジェクト.run(opノード) → opノード.eval()
</div>

**<font color="red">※InteractiveSessionオブジェクトを生成する</font>**
```python
__init__(
    target='',
    graph=None,
    config=None
)
```

| 引数名 | 説明 |
|:--------|:-------|
| `target` | オプション。接続先の実行エンジンを指定する。 |
| `graph` | オプション。特に使用する計算グラフがある場合に指定する。 |
| `config` | オプション。セッションの構成オプションを指定する。 |

 - 「セッションにテンソルオブジェクトを渡す必要がない」
 - Jupiter Notebookのような対話型(インタラクティブモード)の実行環境において、セッションを気にすることなく、コーディングに集中できるように用意された機能

In [8]:
import tensorflow as tf

# データフローグラフの作成
rate = tf.constant(0.08, dtype=tf.float32)
price = tf.Variable(1000.0, dtype=tf.float32)
tax = tf.multiply(rate, price)
price_tax = tf.add(tax, price)
update = tf.assign(price, price_tax)

# セッション部
sess = tf.InteractiveSession() # インタラクティブモードでセッション開始
sess.run(tf.global_variables_initializer())
print('rate = ', rate.eval())
print('price = ', price.eval())
update.eval()
print('price(update) = ', price.eval())

sess.close() # セッションを閉じる

rate =  0.08
price =  1000.0
price(update) =  1080.0


## with文でセッション全体をラップする

 - メリット : セッション部とデータフローグラフ作成部を分け、セッションの処理を関数として定義することでわかりやすくする.

In [9]:
import tensorflow as tf

# データフローグラフの作成
rate = tf.constant(0.08, dtype=tf.float32)
price = tf.Variable(1000.0, dtype=tf.float32)
tax = tf.multiply(rate, price)
price_tax = tf.add(tax, price)
update = tf.assign(price, price_tax)

# セッションの処理を実行する関数
def run_session():
    sess = tf.Session()    # セッションスタート
    # すべての変数をまとめて初期化
    sess.run(tf.global_variables_initializer())
    print('rate = ', sess.run(rate))
    print('price = ', sess.run(price))
    sess.run(update)    # updateノードを実行
    print('price(update) = ', sess.run(price))
    sess.close()    # セッションクローズ

# セッションを実行
run_session()

rate =  0.08
price =  1000.0
price(update) =  1080.0


### `with`文でセッション全体をラップする

```python
with オブジェクトを生成するコード as 変数:
    オブジェクトを用いた処理
```

In [11]:
import tensorflow as tf

# データフローグラフの作成
rate = tf.constant(0.08, dtype=tf.float32)
price = tf.Variable(1000.0, dtype=tf.float32)
tax = tf.multiply(rate, price)
price_tax = tf.add(tax, price)
update = tf.assign(price, price_tax)

# セッションの処理
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print('rate = ', sess.run(rate))
    print('price = ', sess.run(price))
    sess.run(update)
    print('price(update) = ', sess.run(price))

rate =  0.08
price =  1000.0
price(update) =  1080.0


<div style="text-align:center">
    <b>TensorFlowの演算子</b>
</div>

| 演算子 | 説明 |
|:--------|:-------|
| `tf.add(x, y)` | 同じ型のTensorを足し算する($x + y$)。 |
| `tf.subtract(x, y)` | 同じ型のTensorを引き算する($x - y$)。 |
| `tf.multiply(x, y)` | 2つのTensorに対して行列の積を求める。 |
| `tf.pow(x, y)` | $x$の要素ごとに$y$乗する($x^{y}$)。 |
| `tf.exp(x)` | $\mathrm {exp}\left( x \right)$ |
| `tf.sqrt(x)` | $x$の平方根を求める。 |
| `tf.div(x, y)` | $x$と$y$の要素ごとの徐算を行う。 |
| `tf.truediv(x, y)` | float型の引数をキャストする以外は、`div()`と同じ。 |
| `tf.floordiv(x, y)` | 求めた値を整数に丸める以外は、`div()`と同じ。 |
| `tf.mod(x, y)` | 要素ごとの剰余を求める。 |

# `tf.placeholder`(プレースホルダー)
定数(Constant)や変数(Variable)は、宣言時に初期値を設定する。ただ、処理によっては実際にプログラムを実行してみないことには値が決まらないこともある。そようなときに使うのが、`tf.placeholder`。

```python
tf.placeholder(
    dtype,
    shape=None,
    name=None
)
```

| 引数名 | 説明 |
|:--------|:------|
| `dtype` | 要素の型 |
| `shape` | オプション。要素の形状を指定。 |
| `name` | オプション。オペレーションの名前。 |

## 実行時に確定する値をプレースホルダーで保持する
例 : 「指数平滑法(しすうへいかつほう)」を使って時系列データから将来値を予測する処理を行う。

指数平滑法:
$$
\begin{eqnarray}
\hat {y}_{t} & = & \alpha y_{t-1} + (1 - \alpha) \hat {y}_{t-1}\\
& = & \hat {y}_{t-1} + \alpha \left( y_{t-1} - \hat {y}_{t-1} \right)
\end{eqnarray}
$$

 - $\hat {y}_{t}$ : $t$の予測値
 - $y_{t}$ : 直前($t - 1$)の実測値
 - $\hat {y}_{t-1}$ : 直前($t-1$)の予測値
 - $0 < \alpha < 1$

In [12]:
import tensorflow as tf
import numpy as np

# ---データフローグラフの作成---
nd_data = np.random.normal(10, 1, 100)    # 平均10,標準1の正規分布に従う乱数を100個生成するノード

nd_alpha = tf.constant(0.05)    # 定数αノード(値0.05)
nd_measured = tf.placeholder(tf.float32)    # 実測値を保持するプレースホルダーのノード
val_predicted = tf.Variable(0.0) # 予測値を保持する変数(初期値0.0)

nd_init = tf.global_variables_initializer()    # 変数を初期化するノード

# 指数平滑法による予測値を求めるノード
nd_predict = val_predicted + nd_alpha * (nd_measured - val_predicted)

# ---セッションの処理---
with tf.Session() as sess:
    sess.run(nd_init)    # initノードを実行して変数を初期化
    for i in range(len(nd_data)):    # 時系列データを抽出して予測値を求める
        predict = sess.run(
            nd_predict,    # 指数平滑法のop_predictノードを実行
            feed_dict={
                nd_measured: nd_data[i]   # プレースホルダーに時系列データをセット
                }
            )
        sess.run(
            tf.assign(
                val_predicted,    # 変数val_predictedを予測値に更新する
                predict    # 指数平滑法による予測値
            ))
        print(
            '(measured)', nd_data[i],  # 実測値を出力
            '(predicted)', predict     # 予測値を出力
            )

(measured) 9.346932570793134 (predicted) 0.46734664
(measured) 10.366985388078238 (predicted) 0.9623286
(measured) 9.566187865499398 (predicted) 1.3925216
(measured) 8.767875092900873 (predicted) 1.7612892
(measured) 10.990154347942353 (predicted) 2.2227325
(measured) 10.36930543711687 (predicted) 2.6300611
(measured) 10.596902354515937 (predicted) 3.0284033
(measured) 9.67530567095879 (predicted) 3.3607483
(measured) 8.959198803571688 (predicted) 3.6406708
(measured) 8.699302080035249 (predicted) 3.8936024
(measured) 12.369214799469734 (predicted) 4.317383
(measured) 9.699843347142904 (predicted) 4.586506
(measured) 10.230180595714181 (predicted) 4.8686895
(measured) 9.99635225471173 (predicted) 5.1250725
(measured) 9.564458473192829 (predicted) 5.3470416
(measured) 11.050979889706088 (predicted) 5.6322384
(measured) 9.197160528831416 (predicted) 5.8104844
(measured) 10.755318115029366 (predicted) 6.057726
(measured) 8.665273093095179 (predicted) 6.188103
(measured) 9.541500808931099 